# Metadata collection with ZnTrack

ZnTrack allows for the collection of some metadata.
One example is measuring the execution time of Nodes or even methods inside the Nodes easily.
This can be achieved by using the `@TimeIt` decorator which is shown in the following example.

In [1]:
from zntrack import Node, zn, config
from zntrack.metadata import TimeIt
from time import sleep

config.nb_name = "05_metadata.ipynb"

In [2]:
from zntrack.utils import cwd_temp_dir

temp_dir = cwd_temp_dir()

In [3]:
!git init
!dvc init

Initialized empty Git repository in /tmp/tmp4us1bcfh/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [4]:
class SleepNode(Node):
    metadata = zn.metadata()

    @TimeIt
    def run(self):
        self.sleep_1s()
        self.sleep_2s()

    @TimeIt
    def sleep_1s(self):
        sleep(1)

    def sleep_2s(self):
        sleep(2)

In [5]:
SleepNode().write_graph(run=True)

2022-04-29 15:25:27,153 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.
2022-04-29 15:25:31,612 (WARNING): Running DVC command: 'dvc run -n SleepNode ...'


In [6]:
!dvc metrics show

ore>Path                                   metadata.run:timeit    metadata.sleep_1s:timeit
nodes/SleepNode/metrics_no_cache.json  3.00312                1.00053


We can also time a sinlge function multiple times, using the following example:

In [7]:
class SleepNodeMulti(Node):
    metadata = zn.metadata()

    @TimeIt
    def run(self):
        self.sleep(1.0)
        self.sleep(0.8)
        self.sleep(1.2)

    @TimeIt
    def sleep(self, time):
        sleep(time)

In [8]:
SleepNodeMulti().write_graph(run=True)

2022-04-29 15:25:43,644 (WARNING): Running DVC command: 'dvc run -n SleepNodeMulti ...'


In [9]:
!dvc metrics show

ore>Path                                        metadata.run:timeit    metadata.sleep:timeit.mean    metadata.sleep:timeit.std    metadata.sleep_1s:timeit
nodes/SleepNode/metrics_no_cache.json       3.00312                -                             -                            1.00053
nodes/SleepNodeMulti/metrics_no_cache.json  3.00455                1.00099                       0.20019                      -


One can also access the metrics directly within Python. This is possible, because they are just another `zn.metrics` which is automatically added when using one of the given metadata decorators. This also gives access to the individual run times which are hidden when using `dvc metrics show`.

In [10]:
SleepNodeMulti.load().metadata

{'sleep:timeit': {'values': [1.0008997917175293,
   0.8008444309234619,
   1.2012319564819336],
  'mean': 1.0009920597076416,
  'std': 0.20019377872637664},
 'run:timeit': 3.0045523643493652}

In [11]:
temp_dir.cleanup()